In [1]:
import pandas as pd
import numpy as np
from Data_Processing import DataProcessing
from sklearn.model_selection import train_test_split
import joblib

from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [2]:
pop = pd.read_csv('../Data/population.csv')
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [3]:
X, y, test = DataProcessing(train, test, pop)
#pca = PCA(n_components=200)
#X = pca.fit_transform(X)
y = y.ravel()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [5]:
X_train.shape

(430866, 78)

### Baseline

In [6]:
# rf = RandomForestRegressor(n_jobs=-1)

# rf.fit(X_train, y_train)

In [7]:
# y_true = y_test
# y_pred = rf.predict(X_test)

# mean_squared_error(y_true, y_pred)

## XGBoost

In [8]:
import xgboost as xgb
from bayes_opt import BayesianOptimization

In [9]:
#Importing necessary libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#Converting the dataframe into XGBoost’s Dmatrix object
dtrain = xgb.DMatrix(X_train, label=y_train)

#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, n_estimators, learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'rmse'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=50, nfold=5)
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (1,8),
                                            'gamma': (0,0.1),
                                            'n_estimators':(10,100),
                                            'learning_rate':(0,0.1),
                                            })

In [10]:
xgb_bo.maximize(n_iter=10, init_points=8, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------
[18:40:49] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:40:49] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:40:49] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, 

In [11]:
params = xgb_bo.max['params']
print(params)

{'gamma': 0.09641319527701114, 'learning_rate': 0.09094321316167273, 'max_depth': 4.830870722053943, 'n_estimators': 44.6047601950532}


In [12]:
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])

In [13]:
from xgboost import XGBRegressor
xgb1_model = XGBRegressor(**params).fit(X_train, y_train)

In [16]:
y_true = y_test
y_pred = xgb1_model.predict(X_test)

mean_squared_error(y_true, y_pred)

0.6557108644198254

In [22]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')

train_zips = train_df['postcode']

y_true = y
y_pred = xgb1_model.predict(X)

df = pd.DataFrame({'postcode': train_zips, 'y_true': y_true, 'y_pred': y_pred}, columns=['postcode', 'y_true', 'y_pred'])
df = df.groupby('postcode').mean()

df

,y_true,y_pred
postcode,,
AB10 1AU,1.500000,1.407120
AB10 1BD,1.200000,1.410336
AB10 1BW,2.000000,1.421243
AB10 1HT,1.000000,1.498882
AB10 1NJ,1.428571,1.441475
...,...,...
ZE2 9QJ,1.500000,1.421421
ZE2 9QS,1.600000,1.467412
ZE2 9RE,1.500000,1.391122


In [23]:
y_pred = xgb1_model.predict(test)
#y_pred = model.predict(pca.transform(test))

test_zips = test_df['postcode']
test_df = pd.DataFrame({'postcode': test_zips, 'Accident_risk_index': y_pred.flatten()}, columns=['postcode', 'Accident_risk_index'])

submission = test_df.groupby('postcode').mean().reset_index()

submission.to_csv('../Submissions/XGBoost.csv', index=False)
submission

,postcode,Accident_risk_index
0,AB10 1AU,1.402500
1,AB10 1PG,1.379428
2,AB10 1TT,1.565190
3,AB10 1YP,1.415460
4,AB10 6LQ,1.376166
...,...,...
49767,ZE2 9LZ,1.344388
49768,ZE2 9RE,1.377919
49769,ZE2 9RJ,1.297642
49770,ZE2 9SB,1.543372


# XGB 2

In [ ]:
# #Specify which dataset and which metric should be used for early stopping.
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# es = xgb.callback.EarlyStopping(
#     rounds=3,
#     save_best=True,
#     maximize=False,
#     data_name="Valid",
#     metric_name="rmse",
# )

# boost = xgb.train(
#     {'objective': 'reg:squarederror',
#      'eval_metric': ['rmse'],
#      'tree_method': 'hist'}, dtrain,
#     evals=[(dtrain, 'Train'), (dtest, 'Valid')],
#     num_boost_round=1000,
#     callbacks=[es],
#     verbose_eval=True)

In [ ]:
# y_true = y_test
# y_pred = boost.predict(dtest)

# mean_squared_error(y_true, y_pred)

In [ ]:
# train_df = pd.read_csv('../Data/train.csv')
# test_df = pd.read_csv('../Data/test.csv')

# train_zips = train_df['postcode']

# y_true = y
# y_pred = boost.predict(xgb.DMatrix(X))

# df = pd.DataFrame({'postcode': train_zips, 'y_true': y_true, 'y_pred': y_pred}, columns=['postcode', 'y_true', 'y_pred'])
# df = df.groupby('postcode').mean()

# df

In [ ]:
# y_pred = boost.predict(xgb.DMatrix(test))
# #y_pred = model.predict(pca.transform(test))

# test_zips = test_df['postcode']
# test_df = pd.DataFrame({'postcode': test_zips, 'Accident_risk_index': y_pred.flatten()}, columns=['postcode', 'Accident_risk_index'])

# submission = test_df.groupby('postcode').mean().reset_index()

# submission.to_csv('../Submissions/XGBoost.csv', index=False)
# submission

In [ ]:
# y.shape

In [ ]:
# train_zips